In [1]:
####################################### PART ONE #######################################
import tensorflow as tf
import numpy as np
from PIL import Image
import datetime

num_classes = 200
limit = 2
num_classes = limit
batch_size = 10
epochs = 10
# out_channels = [35, 50, 25, 3]
out_channel_sizes = [5, 15, 15, 3]

RUN_PREFIX = datetime.datetime.fromtimestamp(datetime.datetime.now().timestamp()).strftime('%Y-%m-%d-%H_%M_%S')


def load(filename):
    file = open(filename, "r") 
    image_names = file.readlines()
    images = []
    labels = []
    for name in image_names:
        label = int(name[:3])
        if label <= limit:
            im = Image.open("images/" + name.rstrip('\n'))
            H, W = im.size
            pixels = list(im.getdata())
            if not type(pixels[0]) is int:
                # todo: right now we are discarding transparent images
                image = np.array([comp for pixel in pixels for comp in pixel]).reshape(-1, H, W, 3)
                images.append(image)
                # zero-index the label
                labels.append(label - 1)
    return images, labels

images_train, labels_train = load('train.txt')
images_test, labels_test = load('test.txt')

print(len(images_train))
print(len(images_test))



60
60


In [7]:
####################################### PART TWO #######################################

# todo: use tf.contrib.layers.conv2d for 3D filter
def conv_relu(input_image, kernel_shape, bias_shape, stride = 2):
    strides = [1, stride, stride, 1]
    weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())
    # variable_summaries(weights)
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    # variable_summaries(biases)
    conv = tf.nn.conv2d(input_image, weights, strides=strides, padding='SAME')

    return tf.nn.relu(conv + biases)

def conv_layers(input_image):
    # Variables created here will be named "convX/weights", "convX/biases".
    with tf.variable_scope("conv1"):
        out_channels = out_channel_sizes[0]
        output = conv_relu(input_image, [5, 5, 3, out_channels], [out_channels], stride=1)
    # with tf.variable_scope("conv2"):
    #     old_out_channels = out_channels
    #     out_channels = out_channel_sizes[1]
    #     output = conv_relu(output, [5, 5, old_out_channels, out_channels], [out_channels])
    # with tf.variable_scope("conv3"):
    #     old_out_channels = out_channels
    #     out_channels = out_channel_sizes[2]
    #     output = conv_relu(output, [5, 5, old_out_channels, out_channels], [out_channels], stride=1)
    with tf.variable_scope("conv2"):
        old_out_channels = out_channels
        out_channels = out_channel_sizes[3]
        return conv_relu(output, [5, 5, old_out_channels, out_channels], [out_channels])


# Modified from RikHeijdens on https://github.com/tensorflow/tensorflow/issues/6011
def spp_layer(image, dimensions=[3, 2, 1]):
    # todo: fix this
    # if tf.less(tf.shape(image)[1], dimensions[0] ** 2) is True:
    #     return None
    # if tf.less(tf.shape(image)[2], dimensions[0] ** 2) is True:
    #     return None
    pool_list = []
    for pool_dim in dimensions:
        pool_list += max_pool_2d_nxn_regions(image, pool_dim)
    return tf.concat(pool_list, axis=1)

def max_pool_2d_nxn_regions(inputs, output_size):
    inputs_shape = tf.shape(inputs)
    h = tf.cast(tf.gather(inputs_shape, 1), tf.int32)
    w = tf.cast(tf.gather(inputs_shape, 2), tf.int32)

    result = []
    n = output_size
    for row in range(output_size):
        for col in range(output_size):
            # start_h = floor(row / n * h)
            start_h = tf.cast(tf.floor(tf.multiply(row / n, tf.cast(h, tf.float32))), tf.int32)
            # end_h = ceil((row + 1) / n * h)
            end_h = tf.cast(tf.ceil(tf.multiply((row + 1) / n, tf.cast(h, tf.float32))), tf.int32)
            # start_w = floor(col / n * w)
            start_w = tf.cast(tf.floor(tf.multiply(col / n, tf.cast(w, tf.float32))), tf.int32)
            # end_w = ceil((col + 1) / n * w)
            end_w = tf.cast(tf.ceil(tf.multiply((col + 1) / n, tf.cast(w, tf.float32))), tf.int32)
            pooling_region = inputs[:, start_h:end_h, start_w:end_w, :]
            pool_result = tf.reduce_max(pooling_region, axis=(1, 2))
            result.append(pool_result)
    return result

# todo: might be able to move this into session
def fc_layer(image):
#     return image
    return tf.contrib.layers.fully_connected(image, num_classes, activation_fn=None, scope="fc")

# def get_scope_variable(scope_name, var, shape=None, dtype=tf.float64):
#     with tf.variable_scope(scope_name) as scope:
#         try:
#             v = tf.get_variable(var, shape, dtype=dtype)
#         except ValueError:
#             scope.reuse_variables()
#             v = tf.get_variable(var, dtype=dtype)
#     return v

LOGITS_COLLECTION = 'LOGITS'
LOGIT_LABELS_COLLECTION = 'LOGIT-LABELS'
graph = tf.Graph()
with graph.as_default():

    image_placeholders = []
    label_placeholders = []

    # fc_reuse = False
    with tf.variable_scope("network") as scope:

        for i in range(batch_size):
            image = tf.placeholder(tf.float32, shape=(1,None,None,3), name='image_{}'.format(i))
            image_placeholders.append(image)
            label = tf.placeholder(tf.int64, shape=(), name='label_{}'.format(i))
            label_placeholders.append(label)

            logit = tf.to_float(image)
            logit = conv_layers(logit)
            logit = spp_layer(logit)

            if not logit is None:
                logit = fc_layer(logit)
                logit = tf.reshape(logit, [-1])
                x = tf.identity(logit, name='test_{}'.format(i))
                y = tf.identity(label, name='test_label_{}'.format(i))
                tf.add_to_collection(LOGITS_COLLECTION, x)
                tf.add_to_collection(LOGIT_LABELS_COLLECTION, y)
                # tf.Variable(x, expected_shape=[1, 2], collections=[LOGITS_COLLECTION, tf.GraphKeys.GLOBAL_VARIABLES])
                # tf.Variable(y, expected_shape=[], collections=[LOGIT_LABELS_COLLECTION, tf.GraphKeys.GLOBAL_VARIABLES])
                # tf.Variable(y, expected_shape=[])
                # tf.Variable(x, expected_shape=[1, 2], name='logit_{}'.format(i))
                # tf.Variable(label, expected_shape=[1], name='logit_label_{}'.format(i))
                # import ipdb; ipdb.set_trace()
                # fc_reuse = True

            scope.reuse_variables()
#             print('+++++++++++++++++++++++++++++++++')
#             [print(v.name) for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='network')]
        
    print('==================================')
    [print(v.name) for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)]
    # [[print(v.name) for v in tf.get_collection(key)] for key in graph.get_all_collection_keys()]
    logits = tf.stack(tf.get_collection(LOGITS_COLLECTION))
    logit_labels = tf.stack(tf.get_collection(LOGIT_LABELS_COLLECTION))
        
    # logits = tf.stack([get_scope_variable('network', 'image_{}'.format(i), [1, 2]) for i in range(batch_size)])
    # logit_labels = tf.stack([get_scope_variable('network', 'label_{}'.format(i), [1], tf.int64) for i in range(batch_size)])

    print(logits.shape)
    # print(logit_labels.shape)
        
#     logits = tf.convert_to_tensor(logits)
#     logit_labels = tf.convert_to_tensor(logit_labels)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=logit_labels))# + tf.losses.get_regularization_loss()
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        opt = tf.train.MomentumOptimizer(0.001, 0.9).minimize(loss)
    correct = tf.equal(tf.argmax(logits, -1), logit_labels)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    print("Total number of variables used ", np.sum([v.get_shape().num_elements() for v in tf.trainable_variables()]))

####################################### PART THREE #######################################
# saver = tf.train.Saver(var_list=tf.trainable_variables())
sess = tf.Session(graph=graph)
# with graph.as_default():
#     print('.here1')
    
#     print('.here2')
LOG_DIR = 'log/{}'.format(RUN_PREFIX)
# sess.run(tf.global_variables_initializer())


with graph.as_default(), sess.as_default():
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOG_DIR)
    writer.add_graph(sess.graph)
    for epoch in range(epochs):
        np.random.seed(epoch)
        np.random.shuffle(images_train)
        np.random.seed(epoch)
        np.random.shuffle(labels_train)
        accuracy_vals, loss_vals = [], []
        for i in range(0, len(images_train) - batch_size + 1, batch_size):
            batch_images, batch_labels = images_train[i:i + batch_size], labels_train[i:i + batch_size]

    #         batch_images, batch_labels = images_train[i:i + 2], labels_train[i:i + 2]
        
            # todo: this is not very good... (probably replace with 1 x 1 x 1 x 1 when I implement SPP filter)
            if batch_size - len(batch_images) > 0:
                # print('testing diff: %d'%(batch_size - len(batch_images)))
                for j in range(len(batch_images), batch_size):
                    batch_images.append(images_train[j - len(batch_images)])
                    batch_labels.append(labels_train[j - len(batch_images)])

            fd = {**{k: v for k, v in zip(image_placeholders, batch_images)}, **{k: v for k, v in zip(label_placeholders, batch_labels )}}

            accuracy_val, loss_val, _ = sess.run([accuracy, loss, opt], feed_dict=fd)
            accuracy_vals.append(accuracy_val)
            loss_vals.append(loss_val)
        val_correct = []
        for i in range(0, len(images_test), batch_size):
    #         batch_images, batch_labels = images_test[i:i + batch_size], labels_test[i:i + batch_size]
            batch_images, batch_labels = images_test[i:i + batch_size], labels_test[i:i + batch_size]
            
            # todo: this is not very good... (probably replace with 1 x 1 x 1 x 1 when I implement SPP filter)
            if batch_size - len(batch_images) > 0:
                # print('training diff: %d'%(batch_size - len(batch_images)))
                for j in range(len(batch_images), batch_size):
                    batch_images.append(images_test[j - len(batch_images)])
                    batch_labels.append(labels_test[j - len(batch_images)])
                
            fd = {**{k: v for k, v in zip(image_placeholders, batch_images)}, **{k: v for k, v in zip(label_placeholders, batch_labels )}}
            c = sess.run(correct, feed_dict=fd)
            val_correct.extend(c)

        # s = tf.Summary()
        # s.value.add(simple_value=np.mean(accuracy_vals), tag="accuracy")
        # s.value.add(simple_value=np.mean(loss_vals), tag="loss")
        # s.value.add(simple_value=np.mean(val_correct), tag="correct")
        # writer.add_summary(s, epoch)
        # writer.flush()
        # saver.save(sess, path.join(RUN_PREFIX, 'network.ckpt'), global_step=epoch)
        print('[%3d] Accuracy: %0.3f  \t  Loss: %0.3f  \t  validation accuracy: %0.3f'%(epoch, np.mean(accuracy_vals), np.mean(loss_vals), np.mean(val_correct)))


# sess.run([accuracy], feed_dict={image: images_train[0]})
# batch_images = images_train[:batch_size]
# batch_labels = labels_train[:batch_size]
# fd = {**{i: d for i, d in zip(image_placeholders, batch_images)}, **{i: d for i, d in zip(label_placeholders, batch_labels )}}
# accuracy_val, loss_val, _  = sess.run([accuracy, total_loss, opt], feed_dict=fd)
# print(accuracy_val)
# print(loss_val)
# print(logits[0])

# batch_images = images_train[batch_size:batch_size+batch_size]
# batch_labels = labels_train[batch_size:batch_size+batch_size]
# accuracy_val, loss_val, _  = session.run([accuracy, total_loss, opt], feed_dict=fd)
# print(accuracy_val)
# print(loss_val)

# print(logits[0])
print(len(tf.trainable_variables()))

network/conv1/weights:0
network/conv1/biases:0
network/conv2/weights:0
network/conv2/biases:0
network/conv3/weights:0
network/conv3/biases:0
network/conv4/weights:0
network/conv4/biases:0
network/fc/weights:0
network/fc/biases:0
network/logit_0:0
network/logit_label_0:0
network/logit_1:0
network/logit_label_1:0
network/logit_2:0
network/logit_label_2:0
network/logit_3:0
network/logit_label_3:0
network/logit_4:0
network/logit_label_4:0
network/logit_5:0
network/logit_label_5:0
network/logit_6:0
network/logit_label_6:0
network/logit_7:0
network/logit_label_7:0
network/logit_8:0
network/logit_label_8:0
network/logit_9:0
network/logit_label_9:0
(10, 1, 2)
(10, 1)
Total number of variables used  9184


InvalidArgumentError: You must feed a value for placeholder tensor 'network/image_0' with dtype float and shape [1,?,?,3]
	 [[Node: network/image_0 = Placeholder[dtype=DT_FLOAT, shape=[1,?,?,3], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'network/image_0', defined at:
  File "/Users/michaelscaria/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/michaelscaria/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/michaelscaria/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/michaelscaria/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/michaelscaria/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/michaelscaria/anaconda/l